## Scikit-Learn PCA
### Using BREASTCANCER_VIEW from DWC. This view has 569 records

## Install fedml_gcp package

In [1]:
pip install fedml_gcp

Processing ./fedml_gcp-1.0.0-py3-none-any.whl
  Using cached google-3.0.0-py2.py3-none-any.whl (45 kB)
  Using cached hdbcli-2.10.13-cp34-abi3-manylinux1_x86_64.whl (11.7 MB)
  Using cached beautifulsoup4-4.10.0-py3-none-any.whl (97 kB)
  Using cached soupsieve-2.2.1-py3-none-any.whl (33 kB)
  Attempting uninstall: soupsieve
    Found existing installation: soupsieve 2.2.1
    Uninstalling soupsieve-2.2.1:
      Successfully uninstalled soupsieve-2.2.1
  Attempting uninstall: beautifulsoup4
    Found existing installation: beautifulsoup4 4.10.0
    Uninstalling beautifulsoup4-4.10.0:
      Successfully uninstalled beautifulsoup4-4.10.0
  Attempting uninstall: hdbcli
    Found existing installation: hdbcli 2.10.13
    Uninstalling hdbcli-2.10.13:
      Successfully uninstalled hdbcli-2.10.13
  Attempting uninstall: google
    Found existing installation: google 3.0.0
    Uninstalling google-3.0.0:
      Successfully uninstalled google-3.0.0
  Attempting uninstall: fedml-gcp
    Found ex

## Import Libraries

In [2]:
from fedml_gcp import DwcGCP
import numpy as np
import pandas as pd

## Create DwcGCP Instance to access class methods and train model

It is expected that the bucket name passed here already exists in Cloud Storage.

In [3]:
dwc = DwcGCP(project_name='example-project',
                 bucket_name='<bucket-name>')

### Create tar bundle of script folder so GCP can use it for training

Before running this cell, please ensure that the script package has all the necessary files for a training job.

In [4]:
dwc.make_tar_bundle('DimReduction.tar.gz', 'Dimensionality-Reduction', 'dim-red/train/DimReduction.tar.gz')


File DimReduction.tar.gz uploaded to dim-red/train/DimReduction.tar.gz.


### Create tar bundle of predictor script folder so GCP can use it for inferencing

Before running this cell, please ensure that the predictor package has all the necessary files for a training job.

In [5]:
dwc.make_tar_bundle('DimReductionPredictor.tar.gz', 'Dimensionality-Reduction-Predictor', 'dim-red/prediction/DimReductionPredictor.tar.gz')


File DimReductionPredictor.tar.gz uploaded to dim-red/prediction/DimReductionPredictor.tar.gz.


### Train Model

GCP takes in training inputs that are specific to the training job and the environment needed.

In the training inputs, we are the python module. This is the module that your script package is named, and it references the task.py file inside the script package.

We are also passing args which hold the table name to get data from. Before running the following cell, you should have a config.json file in the script package with the specified values to allow you to access to DWC.

You should also have the follow view `BREASTCANCER_VIEW` created in your DWC. To gather this data, please refer to https://www.kaggle.com/uciml/breast-cancer-wisconsin-data

In [6]:
training_inputs = {
    'scaleTier': 'BASIC',
    'packageUris': ['gs://<bucket-name>/dim-red/train/DimReduction.tar.gz'],
    'pythonModule': 'trainer.task',
    'args': ['--table_name', 'BREASTCANCER_VIEW',
             '--num_components', '3',
            '--bucket_name', '<bucket-name>'],
    'region': 'us-east1',
    'jobDir': 'gs://<bucket-name>',
    'runtimeVersion': '2.5',
    'pythonVersion': '3.7',
    'scheduling': {'maxWaitTime': '3600s', 'maxRunningTime': '7200s'}
}

In [7]:
dwc.train_model('dim_red_final_train2', training_inputs)

Training Job Submitted Succesfully
Job status for sap-ti-ci-sce.dim_red_final_train2:
    state : QUEUED


### Deploy Model

In [ ]:
dwc.deploy_to_kyma('example@project-name.gserviceaccount.com', 'key.json', 'custom-kyma-deploy', custom_predictor='Dimensionality-Reduction-Predictor/predictor.py',  model_location='Dimensionality-Reduction/model/model.pkl')